In [1]:
import pandas as pd
import json
import numpy as np

## Load Data Files

In [2]:

metadata_df = pd.read_csv('data/SDP.csv')
print(f"Loaded {len(metadata_df)} metadata records")
metadata_df.head()

Loaded 875 metadata records


,id,cell_type,ctrl_ids,curator,drug_name,drugbank_id,geo_id,organism,pert_ids,platform,pubchem_cid,smiles,version
0,drug:3639,Bone marrow Sca+ SP hematopoeitic stem cells (...,GSM26734|GSM26735,cadimo,Fluorouracil,DB00544,GSE1559,mouse,GSM26744|GSM26745,GPL81,3385.0,C1=C(C(=O)NC(=O)N1)F,1.0
1,drug:3499,vastus lateralis muscle,GSM801199|GSM801201|GSM801202|GSM801204|GSM801...,MicrotaskManic,Resveratrol,DB02709,GSE32357,human,GSM801198|GSM801200|GSM801203|GSM801205|GSM801...,GPL11532,NaN,Oc1ccc(cc1)/C=C/c1cc(O)cc(c1)O,1.0
2,drug:3292,lymphoblastoid cells,GSM162898|GSM162900|GSM162903,garibr01,Citalopram,DB00215,GSE7036,human,GSM162897|GSM162899|GSM162901,GPL570,2771.0,CN(C)CCCC1(C2=C(CO1)C=C(C=C2)C#N)C3=CC=C(C=C3)F,1.0
3,drug:3638,Bone marrow Sca+ SP hematopoeitic stem cells (...,GSM26734|GSM26735,cadimo,Fluorouracil,DB00544,GSE1559,mouse,GSM26742|GSM26743,GPL81,3385.0,C1=C(C(=O)NC(=O)N1)F,1.0
4,drug:3475,liver,GSM1273512|GSM1273513|GSM1273514|GSM1273515,MicrotaskManic,Ethanol,DB00898,GSE52644,mouse,GSM1273500|GSM1273501|GSM1273502|GSM1273503,GPL1261,702.0,CCO,1.0


In [4]:

with open('data/DP.json', 'r') as f:
    drug_data = json.load(f)

print(f"Loaded {len(drug_data)} drug perturbation records")

Loaded 875 drug perturbation records


## Filter for Human Drugs

In [5]:
# Filter drugs where organism is 'human'
human_drugs = [drug for drug in drug_data if drug.get('organism', '').lower() == 'human']
print(f"Found {len(human_drugs)} human drug records")

Found 450 human drug records


## Define Input Gene Arrays

In [6]:
# Input array 1: Up-regulated genes in disease
up_regulated_genes = [
    "LYZ",
    "CSRP3",
    "SNORD13"
]

# Input array 2: Down-regulated genes in disease
down_regulated_genes = [
    "RNU1-1",
    "GAPDH",
    "RPL8"
    

    
]

## Match Genes with Drug Data

In [22]:
def match_genes_to_drugs(human_drugs, up_regulated_genes, down_regulated_genes):
    
    drug_aggregates = {}
    
    for drug in human_drugs:
        drug_name = drug.get('drug_name', 'Unknown')
        smiles = drug.get('smiles', 'N/A')
        cell_type = drug.get('cell_type', 'N/A')
        idx = drug.get('id', 'N/A')
        
        # Initialize drug entry if not exists
        if drug_name not in drug_aggregates:
            drug_aggregates[drug_name] = {
                'smiles': smiles,
                'up_reg_matches': [],
                'down_reg_matches': [],
                'cell_types': [],
                'drug_ids': []
            }
        
        # Collect cell types and drug IDs
        if cell_type not in drug_aggregates[drug_name]['cell_types']:
            drug_aggregates[drug_name]['cell_types'].append(cell_type)
        if idx not in drug_aggregates[drug_name]['drug_ids']:
            drug_aggregates[drug_name]['drug_ids'].append(idx)
        
        # Search up-regulated genes in drug's down_genes
        down_genes_dict = {gene[0]: gene[1] for gene in drug.get('down_genes', [])}
        for gene in up_regulated_genes:
            if gene in down_genes_dict:
                drug_aggregates[drug_name]['down_reg_matches'].append({
                    'gene': gene,
                    'score': down_genes_dict[gene]
                })
        
        # Search down-regulated genes in drug's up_genes
        up_genes_dict = {gene[0]: gene[1] for gene in drug.get('up_genes', [])}
        for gene in down_regulated_genes:
            if gene in up_genes_dict:
                drug_aggregates[drug_name]['up_reg_matches'].append({
                    'gene': gene,
                    'score': up_genes_dict[gene]
                })
    
    # Convert to results list
    results = []
    for drug_name, data in drug_aggregates.items():
        # Add down-regulation row if matches exist
        if data['down_reg_matches']:
            results.append({
                'drug_name': drug_name,
                'smiles': data['smiles'],
                'regulation_type': 'down',
                'matched_genes': ','.join([m['gene'] for m in data['down_reg_matches']]),
                'regulation_scores': ','.join([f"{m['score']:.4f}" for m in data['down_reg_matches']]),
                'avg_score': np.mean([m['score'] for m in data['down_reg_matches']]),
                'gene_count': len(data['down_reg_matches']),
                'drug_ids': ','.join(data['drug_ids']),
                'cell_types': ','.join(data['cell_types'])
            })
        
        # Add up-regulation row if matches exist
        if data['up_reg_matches']:
            results.append({
                'drug_name': drug_name,
                'smiles': data['smiles'],
                'regulation_type': 'up',
                'matched_genes': ','.join([m['gene'] for m in data['up_reg_matches']]),
                'regulation_scores': ','.join([f"{m['score']:.4f}" for m in data['up_reg_matches']]),
                'avg_score': np.mean([m['score'] for m in data['up_reg_matches']]),
                'gene_count': len(data['up_reg_matches']),
                'drug_ids': ','.join(data['drug_ids']),
                'cell_types': ','.join(data['cell_types'])
            })
    
    return results

In [23]:
# Execute matching
results = match_genes_to_drugs(human_drugs, up_regulated_genes, down_regulated_genes)
print(f"Found {len(results)} drug-regulation rows")

Found 152 drug-regulation rows


## Generate Results Table

In [24]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display results
if len(results_df) > 0:
    print(f"\nTotal rows: {len(results_df)}")
    print(f"Unique drugs: {results_df['drug_name'].nunique()}")
    print(f"\nSample of results:")
    display(results_df.head(20))
else:
    print("No matches found")


Total rows: 152
Unique drugs: 126

Sample of results:


,drug_name,smiles,regulation_type,matched_genes,regulation_scores,avg_score,gene_count,drug_ids,cell_types
0,Resveratrol,Oc1ccc(cc1)/C=C/c1cc(O)cc(c1)O,down,"LYZ,CSRP3,SNORD13,SNORD13,SNORD13,CSRP3","-0.2412,-0.1543,-0.1507,-0.1937,-0.1127,-0.0339",-0.147767,6,"drug:3499,drug:3498,drug:3497,drug:3494,drug:3...","vastus lateralis muscle,PBMC (peripheral blood..."
1,Resveratrol,Oc1ccc(cc1)/C=C/c1cc(O)cc(c1)O,up,"RNU1-1,GAPDH,RPL8,RPL8,RNU1-1,GAPDH,GAPDH,RNU1...","0.1403,0.1318,0.1204,0.0357,0.0213,0.0319,0.01...",0.059455,9,"drug:3499,drug:3498,drug:3497,drug:3494,drug:3...","vastus lateralis muscle,PBMC (peripheral blood..."
2,Citalopram,CN(C)CCCC1(C2=C(CO1)C=C(C=C2)C#N)C3=CC=C(C=C3)F,up,RPL8,0.0138,0.013782,1,drug:3292,lymphoblastoid cells
3,Gefitinib,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,up,GAPDH,0.0191,0.019089,1,drug:3474,PC-3 cells
4,Tretinoin,CC1=C(C(CCC1)(C)C)C=CC(=CC=CC(=CC(=O)O)C)C,down,"SNORD13,LYZ,SNORD13,LYZ,SNORD13,LYZ,LYZ,SNORD13","-0.0958,-0.0221,-0.1298,-0.0222,-0.1198,-0.022...",-0.067497,8,"drug:3233,drug:3232,drug:3231,drug:3237,drug:3...",NB4 APL 9 acute promyelocytic leukemia cells -...
5,Hypochlorous acid,OCl,up,RPL8,0.0534,0.053378,1,"drug:3198,drug:3199,drug:3202,drug:3197,drug:3...",AEC (primary airway epithelial cells) - 6 Hou...
6,4-hydroxytamoxifen,CCC(=C(C1=CC=C(C=C1)O)C2=CC=C(C=C2)OCCN(C)C)C3...,up,RPL8,0.0319,0.031886,1,"drug:3239,drug:3240","MCF7 breast cancer cells - 100nM,MCF7 breast c..."
7,Mycophenolic acid,CC1=C(C(=C(C2=C1COC2=O)O)CC=C(C)CCC(=O)O)OC,down,LYZ,-0.0267,-0.026655,1,drug:3302,No Data
8,Mycophenolic acid,CC1=C(C(=C(C2=C1COC2=O)O)CC=C(C)CCC(=O)O)OC,up,GAPDH,0.0141,0.014080,1,drug:3302,No Data
9,Abiraterone,O[C@H]1CC[C@]2(C(=CC[C@@H]3[C@@H]2CC[C@]2([C@H...,down,SNORD13,-0.0352,-0.035187,1,drug:3300,No Data


## Export Results

In [25]:
# Save results to CSV
if len(results_df) > 0:
    results_df.to_csv('data/drug_gene_matches_unique.csv', index=False)
    print("Results saved to 'drug_gene_matches_unique.csv'")

Results saved to 'drug_gene_matches_unique.csv'


## Summary Statistics

In [26]:
# Verify each drug has at most 2 rows
if len(results_df) > 0:
    print("\nRows per drug:")
    rows_per_drug = results_df.groupby('drug_name').size()
    print(rows_per_drug.value_counts().sort_index())
    


Rows per drug:
1    100
2     26
Name: count, dtype: int64


In [27]:
# Summary by regulation type
if len(results_df) > 0:
    print("\nRegulation Type Summary:")
    reg_summary = results_df.groupby('regulation_type').agg({
        'drug_name': 'count',
        'gene_count': 'sum',
        'avg_score': 'mean'
    }).round(4)
    reg_summary.columns = ['drug_count', 'total_genes', 'avg_score']
    display(reg_summary)


Regulation Type Summary:


,drug_count,total_genes,avg_score
regulation_type,,,
down,34,56,-0.0492
up,118,304,0.0464


In [28]:
# Top drugs by total gene matches
if len(results_df) > 0:
    print("\nTop 20 Drugs by Total Gene Matches:")
    top_drugs = results_df.groupby('drug_name').agg({
        'gene_count': 'sum',
        'avg_score': 'mean'
    }).round(4).sort_values('gene_count', ascending=False)
    top_drugs.columns = ['total_genes', 'overall_avg_score']
    display(top_drugs.head(20))


Top 20 Drugs by Total Gene Matches:


,total_genes,overall_avg_score
drug_name,,
Estradiol,17,0.0172
Resveratrol,15,-0.0442
Cisplatin,13,-0.0410
Doxorubicin,13,0.0081
Azacitidine,9,0.0456
Mln4924,9,0.0348
Decitabine,9,0.0438
Nickel,8,0.0074
Triiodothyronine-[13c6] hydrochloride (t3 thyronine),8,-0.0017


In [29]:
# View specific drug's both rows
if len(results_df) > 0:
    print("\nExample: View both rows for a specific drug")
    sample_drug = results_df['drug_name'].iloc[0]
    print(f"\nShowing entries for: {sample_drug}")
    display(results_df[results_df['drug_name'] == sample_drug])


Example: View both rows for a specific drug

Showing entries for: Resveratrol


,drug_name,smiles,regulation_type,matched_genes,regulation_scores,avg_score,gene_count,drug_ids,cell_types
0,Resveratrol,Oc1ccc(cc1)/C=C/c1cc(O)cc(c1)O,down,"LYZ,CSRP3,SNORD13,SNORD13,SNORD13,CSRP3","-0.2412,-0.1543,-0.1507,-0.1937,-0.1127,-0.0339",-0.147767,6,"drug:3499,drug:3498,drug:3497,drug:3494,drug:3...","vastus lateralis muscle,PBMC (peripheral blood..."
1,Resveratrol,Oc1ccc(cc1)/C=C/c1cc(O)cc(c1)O,up,"RNU1-1,GAPDH,RPL8,RPL8,RNU1-1,GAPDH,GAPDH,RNU1...","0.1403,0.1318,0.1204,0.0357,0.0213,0.0319,0.01...",0.059455,9,"drug:3499,drug:3498,drug:3497,drug:3494,drug:3...","vastus lateralis muscle,PBMC (peripheral blood..."


## Saving Unique Drugs

In [30]:
def process_drugs_to_csv(human_drugs):
   
    results = []
    
    for drug in human_drugs:
        drug_name = drug.get('drug_name', 'Unknown')
        smiles = drug.get('smiles', 'N/A')
        cell_type = drug.get('cell_type', 'N/A')
        idx = drug.get('id', 'N/A')
        
        # Down-regulation row
        down_genes = drug.get('down_genes', [])
        if down_genes:
            results.append({
                'drug_name': drug_name,
                'smiles': smiles,
                'cell_type': cell_type,
                'drug_id': idx,
                'regulation_type': 'down',
                'genes': ','.join([gene[0] for gene in down_genes]),
                'scores': ','.join([f"{gene[1]:.4f}" for gene in down_genes]),
                'gene_count': len(down_genes)
            })
        
        # Up-regulation row
        up_genes = drug.get('up_genes', [])
        if up_genes:
            results.append({
                'drug_name': drug_name,
                'smiles': smiles,
                'cell_type': cell_type,
                'drug_id': idx,
                'regulation_type': 'up',
                'genes': ','.join([gene[0] for gene in up_genes]),
                'scores': ','.join([f"{gene[1]:.4f}" for gene in up_genes]),
                'gene_count': len(up_genes)
            })
    
    return results


all_results = process_drugs_to_csv(human_drugs)
all_df = pd.DataFrame(all_results)

print(f"Total rows (all drugs): {len(all_df)}")

# Filter 
unique_df = all_df.drop_duplicates(subset=['drug_name', 'regulation_type'], keep='first')

print(f"Unique rows: {len(unique_df)}")
print(f"Unique drugs: {unique_df['drug_name'].nunique()}")


all_df.to_csv('data/all_drugs.csv', index=False)
unique_df.to_csv('data/unique_drugs.csv', index=False)

Total rows (all drugs): 900
Unique rows: 346
Unique drugs: 173
